<a href="https://colab.research.google.com/github/ShilpaBombale/Fine-Tuning-Projects/blob/main/FT-Llama-2-7b-on-Azure-AI-Studio/Dataset_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import json
import csv

def generate_unique_detailed_spice_netlist_examples(num_examples):
    examples = set()

    while len(examples) < num_examples:
        circuit_type = random.choice(['low_pass', 'high_pass', 'opamp', 'voltage_source', 'rc_differentiator'])
        freq = round(random.uniform(0.1, 10), 2)
        res = random.randint(1, 100) * 1000
        cap = random.randint(10, 1000)
        gain = random.randint(1, 100)
        voltage = random.randint(1, 20)
        time = random.randint(1, 10)
        res1 = random.randint(10, 100)
        res2 = random.randint(1, 50)
        amplitude = round(random.uniform(0.5, 5), 2)

        if circuit_type == 'low_pass':
            prompt = f"Please generate a spice netlist for a 3rd-order low pass RC filter, whose cut-off frequency of {freq} MHz."
            completion = f"""* 3rd-order low-pass RC filter
* Cut-off frequency: {freq} MHz
* Define the circuit
V1 1 0 DC 0 AC 1 SIN(0 1 {freq}MEG)
* First RC stage
R1 1 2 {res}
C1 2 0 {cap}p
* Second RC stage
R2 2 3 {res}
C2 3 0 {cap}p
* Third RC stage
R3 3 4 {res}
C3 4 0 {cap}p
* Output
Vout 4 0
* Simulation commands
.control
AC DEC 10 10 100MEG
PRINT AC V(4)
PLOT AC V(4)
.endc
.end
"""
        elif circuit_type == 'high_pass':
            prompt = f"Generate spice netlist for RC high pass filter with R={res} ohm and C = {cap} pF"
            completion = f"""* High-pass RC filter
* Cut-off frequency: {freq} MHz
* Define the circuit
V1 1 0 DC 0 AC 1 SIN(0 1 {freq}MEG)
* RC high-pass stage
C1 1 2 {cap}p
R1 2 0 {res}
* Output
Vout 2 0
* Simulation commands
.control
AC DEC 10 10 100MEG
PRINT AC V(2)
PLOT AC V(2)
.endc
.end
"""
        elif circuit_type == 'opamp':
            prompt = f"Generate spice netlist for an OPAMP amplifier with a gain of {gain}"
            completion = f"""* Non-inverting op-amp amplifier
* Gain = {gain}
* Define the circuit
V1 1 0 DC 0 AC 1 SIN(0 1 1k)
* Op-amp model (e.g., LM741)
* .model uA741 opamp (GBW=1.5MEG)
* Non-inverting amplifier configuration
XOPAMP 3 2 6 0 OPAMP
* Feedback network
Rin 2 0 1k
Rf 2 3 {gain - 1}k
* Power supply
Vcc 6 0 DC 15
Vee 0 7 DC -15
* Input
Vin 1 3
* Output
Vout 4 0
* Load
Rload 4 0 10k
* Simulation commands
.control
AC DEC 10 10 100k
PRINT AC V(4)
PLOT AC V(4)
.endc
.subckt OPAMP 1 2 3 4
* Nodes: non-inverting input, inverting input, positive supply, negative supply
E1 5 0 1 2 999MEG
R1 5 3 10
C1 5 3 1p
G1 6 0 5 4 1000MEG
E2 7 6 POLY(2) (6,3) (4,5) 1 0
R2 7 4 100MEG
C2 7 4 10u
D1 4 7 DX
D2 7 4 DX
.model DX D(IS=1E-15)
.ends OPAMP
.end
"""
        elif circuit_type == 'voltage_source':
            prompt = f"Generate a spice netlist for a voltage source of {voltage}V in series to a capacitor of {cap}uF and resistor of {res1} ohm. A resistor of {res2} ohm is connected across the same voltage source. Analyse the transience for {time}ms across the resistor {res2} ohm."
            completion = f"""* Transient Analysis Example
* Voltage Source
V1 1 0 {voltage}V
* Capacitor
C1 1 2 {cap}uF
* Resistors
R1 2 0 {res1}
R2 1 0 {res2}
* Analysis
.TRAN 0.1ms {time}ms
* Output
.PRINT TRAN V(1) V(2) I(R2)
.PROBE
.END
"""
        elif circuit_type == 'rc_differentiator':
            prompt = f"Generate spice netlist for an RC differentiator with input squarewave of {freq}kHz and amplitude {amplitude}V"
            completion = f"""* RC Differentiator Circuit with Square Wave Input
* Voltage Source (Square Wave)
V1 IN 0 PULSE(0 {amplitude} 0 0 0 50us 100us 0)
* Resistor and Capacitor (RC Differentiator)
R1 IN N1 {res}
C1 N1 0 {cap}nF
* Analysis
.TRAN 0.1us 5ms
* Output
.PRINT TRAN V(IN) V(N1)
.PROBE
.END
"""
        example = {"prompt": prompt, "completion": completion}
        examples.add(json.dumps(example))

    return [json.loads(example) for example in examples]

# Generate 20000 unique detailed examples
num_examples = 20000
detailed_generated_examples = generate_unique_detailed_spice_netlist_examples(num_examples)

# Save to JSONL
jsonl_output_file_path = 'generated_spice_netlists_20000.jsonl'
with open(jsonl_output_file_path, 'w') as jsonl_file:
    for example in detailed_generated_examples:
        jsonl_file.write(json.dumps(example) + '\n')


print(f"Generated {num_examples} unique examples and saved to {jsonl_output_file_path}")


Generated 20000 unique examples and saved to generated_spice_netlists_20000.jsonl
